In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# # figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch_tensorrt

from torch.utils.tensorboard import SummaryWriter

import pytorch_quantization
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import quant_modules
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import calib
from tqdm import tqdm

print(pytorch_quantization.__version__)

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.tools import get_cf_args, get_dummy_input, load_config
from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    quantize_tensorrt_transform_pass,
    test_quantize_tensorrt_transform_pass,
    fake_quantize_transform_pass,
    graph_calibration_pass,
    evaluate_fake_quantize_pass,
    fake_quantize_to_trt_pass
)
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model, get_tokenizer

set_logging_verbosity("info")


2.1.3


INFO     Set logging level to info
I0317 19:53:44.704674 140265329104704 logger.py:44] Set logging level to info


In [2]:
batch_size = 8
model_name = "vgg7"
dataset_name = "cifar10"

# batch_size = 1
# model_name = "facebook/opt-125m:patched"
# dataset_name = "cola"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
CHECKPOINT_PATH = "/home/qizhu/Desktop/Work/mase/mase_output/test-accu-0.9332.ckpt"
# CHECKPOINT_PATH = "/home/qizhu/Desktop/Work/mase/mase_output/opt125.ckpt"

model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

INFO     Loaded pytorch lightning checkpoint from /home/qizhu/Desktop/Work/mase/mase_output/test-accu-0.9332.ckpt
I0317 09:34:58.133078 140293346346816 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from /home/qizhu/Desktop/Work/mase/mase_output/test-accu-0.9332.ckpt


In [4]:
mg = MaseGraph(model=model)
ori_mg = MaseGraph(model=model)

# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
# _ = model(**dummy_in)


mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

In [5]:
pass_args = {
    "by": "name",
    "default": {"config": {"name": None}},
}

mg = fake_quantize_transform_pass(mg, pass_args)
pass_args_eval = {
    "data_module": data_module,
}

mg = evaluate_fake_quantize_pass(mg, pass_args_eval)

Average execute time for one batch: 1.96ms
Total accuracy: 90.20%


In [5]:
pass_args = {
    "by": "name",
    "default": {"config": {"name": None}},
    "feature_layers_0": {
        "config": {
            "name": "int",
            "input": {
                "precesion": 8,
                "calibrator": "max",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "histogram",
                "quantize_axis": None,
            },
        }
    },    
    "classifier_0": {
        "config": {
            "name": "fp16",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
    "classifier_1": {
        "config": {
            "name": "fp16",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
    "classifier_2": {
        "config": {
            "name": "fp16",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
    "classifier_3": {
        "config": {
            "name": "fp16",
            "input": {
                "precesion": 8,
                "calibrator": "histogram",
                "quantize_axis": None,
            },
            "weight": {
                "calibrator": "max",
                "quantize_axis": None,
            },
        }
    },
}

mg = fake_quantize_transform_pass(mg, pass_args)


In [6]:
pass_args_calibrate = {
    "calibrator": "percentile",
    "percentiles": [99],
    "data_module": data_module,
    "num_batches": 100,
}

In [12]:
for node in mg.fx_graph.nodes:
    print(node.type)

<class 'torch.Tensor'>
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
<class 'torch.Tensor'>


In [7]:
graph_calibration_pass(mg,  pass_args_calibrate)
pass_args_eval = {
    "data_module": data_module,
}

mg = evaluate_fake_quantize_pass(mg, pass_args_eval)

RuntimeError: mat1 and mat2 must have the same dtype

In [10]:
mg.model

GraphModule(
  (feature_layers): Module(
    (0): QuantConv2d(
      3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
      (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=4.9597 calibrator=MaxCalibrator scale=1.0 quant)
      (_weight_quantizer): TensorQuantizer(8bit fake per-tensor amax=0.2797 calibrator=HistogramCalibrator scale=1.0 quant)
    )
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_

In [11]:
pass_args = {
    "onnxFile": "onnx_test.onnx",
    "engineFile": "engine_test.plan",
    "dataloader": data_module.test_dataloader,
}
mg = fake_quantize_to_trt_pass(mg, pass_args)

/home/qizhu/miniconda3/envs/mase/lib/python3.10/site-packages/pytorch_quantization-2.1.3-py3.10-linux-x86_64.egg/pytorch_quantization/tensor_quant.py:378: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if min_amax < 0:
/home/qizhu/miniconda3/envs/mase/lib/python3.10/site-packages/pytorch_quantization-2.1.3-py3.10-linux-x86_64.egg/pytorch_quantization/tensor_quant.py:381: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  max_bound = torch.tensor((2.0**(num_bits - 1 + int(unsigned))) - 1.0, device=amax.device)
/home/qizhu

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Succeeded finding ONNX file!
Succeeded parsing .onnx file!
Succeeded building engine!
[ 0]Input -> DataType.FLOAT (-1, 3, 32, 32) (8, 3, 32, 32) input
[ 1]Output-> DataType.FLOAT (-1, 10) (8, 10) output
Succeeded running model in TensorRT!
Average execute time for one batch: 0.09ms
Total accuracy: 86.37%


In [10]:
pass_args_eval = {
    "data_module": data_module,
}

mg = evaluate_fake_quantize_pass(mg, pass_args_eval)

Average execute time for one batch: 1.13ms
Total accuracy: 90.11%


In [11]:
for node in mg.fx_graph.nodes:
    print(node.name)

x
feature_layers_0
feature_layers_1
feature_layers_2
feature_layers_3
feature_layers_4
feature_layers_5
feature_layers_6
feature_layers_7
feature_layers_8
feature_layers_9
feature_layers_10
feature_layers_11
feature_layers_12
feature_layers_13
feature_layers_14
feature_layers_15
feature_layers_16
feature_layers_17
feature_layers_18
feature_layers_19
feature_layers_20
view
classifier_0
classifier_1
classifier_2
classifier_3
last_layer
output


# OPT-125M


In [2]:
wikitext_info = get_dataset_info("wikitext2")
opt = get_model(
    "facebook/opt-125m:patched",
    task="lm",
    dataset_info=wikitext_info,
    pretrained=True,
)
opt_tokenizer = get_tokenizer("facebook/opt-125m:patched")

print(f"prepare data")
# Get data module for dummy inputs
data_module = MaseDataModule(
    name="wikitext2",
    batch_size=2,
    num_workers=os.cpu_count(),
    max_token_len=128,
    tokenizer=opt_tokenizer,
    load_from_cache_file=True,
    model_name="facebook/opt-125m@patched",
)
data_module.prepare_data()
data_module.setup()

/home/qizhu/miniconda3/envs/mase/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


prepare data


In [5]:
model_info = get_model_info("facebook/opt-125m:patched")
cf_args = get_cf_args(model_info=model_info, task="lm", model=opt)

mg = MaseGraph(model=opt, cf_args=cf_args)

dummy_in = get_dummy_input(model_info, data_module=data_module, task="lm")
if len(mg.model.additional_inputs) > 0:
    dummy_in = dummy_in | mg.model.additional_inputs

# Generate graph and initialize metadata
# print(f"init metadata")
mg, _ = init_metadata_analysis_pass(mg, pass_args=None)
# mg, _ = add_common_metadata_analysis_pass(mg, pass_args={"dummy_in": dummy_in})
# mg, _ = add_software_metadata_analysis_pass(mg, None)

# for node in mg.fx_graph.nodes:
#     print(node.meta['mase'].module)


In [7]:
for node in mg.fx_graph.nodes:
    args, kwargs = None, None
    if node.op == "placeholder":
        result = dummy_in[node.name]
        print(result)

tensor(..., device='meta', size=(1, 128), dtype=torch.int64)
tensor(..., device='meta', size=(1, 128), dtype=torch.int64)
None
None
None
tensor(..., device='meta', size=(1, 128), dtype=torch.int64)
False
False
False
True


In [5]:
for node in mg.fx_graph.nodes:
    # if node.op == "call_module":
    print(node.op)

placeholder
placeholder
placeholder
call_function
placeholder
call_function
placeholder
call_function
placeholder
placeholder
call_function
call_function
placeholder
call_function
call_function
placeholder
call_function
call_function
placeholder
call_function
call_function
call_function
call_function
call_method
call_module
call_function
call_function
call_function
call_function
call_function
call_module
call_function
call_module
call_module
call_function
call_function
call_function
call_function
call_function
call_method
call_method
call_module
call_module
call_module
call_module
call_function
call_function
call_function
call_method
call_module
call_module
call_function
call_function
call_function
call_function
call_function
call_method
call_method
call_module
call_module
call_module
call_module
call_function
call_function
call_function
call_method
call_module
call_module
call_function
call_function
call_function
call_function
call_function
call_method
call_method
call_module
call_mod

In [42]:
import time
for i, batch in enumerate(data_module.val_dataloader()):
    # print(i, batch.keys())
    if len(mg.model.additional_inputs) > 0:
        batch = batch | mg.model.additional_inputs
    curr_time = time.time()
    outputs = mg.model(**batch)
    print(f"Batch {i} took {time.time() - curr_time} seconds")
    # print(outputs)

Batch 0 took 0.29590582847595215 seconds
Batch 1 took 0.3456096649169922 seconds
Batch 2 took 0.2067277431488037 seconds
Batch 3 took 0.23676538467407227 seconds
Batch 4 took 0.2209935188293457 seconds
Batch 5 took 0.19042754173278809 seconds
Batch 6 took 0.16930031776428223 seconds
Batch 7 took 0.2097775936126709 seconds
Batch 8 took 0.15608453750610352 seconds
Batch 9 took 0.17119336128234863 seconds
Batch 10 took 0.15328526496887207 seconds
Batch 11 took 0.1912400722503662 seconds
Batch 12 took 0.20645642280578613 seconds
Batch 13 took 0.15901398658752441 seconds
Batch 14 took 0.16031408309936523 seconds
Batch 15 took 0.19242572784423828 seconds
Batch 16 took 0.1795520782470703 seconds
Batch 17 took 0.16906023025512695 seconds
Batch 18 took 0.1770777702331543 seconds
Batch 19 took 0.18641257286071777 seconds
Batch 20 took 0.15935230255126953 seconds
Batch 21 took 0.15850019454956055 seconds
Batch 22 took 0.16030383110046387 seconds
Batch 23 took 0.15561127662658691 seconds
Batch 24 

In [8]:
dummy_in = {
    k: v.cuda() if isinstance(v, torch.Tensor) else v
    for k, v in dummy_in.items()
}

NotImplementedError: Cannot copy out of meta tensor; no data!

In [7]:
pass_args = {
    "precision": 'int8',                                                     # collect weight statistics for linear layers
    "nCalibration": 10,                                                # collect activation statistics for relu layers
    "dummy_in": dummy_in,
    "onnxFile": 'model.onnx',
    "cacheFile": 'model.INT8Cache',  
    "engineFile": 'model.plan'
}
engine = quantize_tensorrt_transform_pass(mg, pass_args)

NotImplementedError: Cannot copy out of meta tensor; no data!